In [ ]:
import os
import re
from datetime import datetime
import math

import tensorflow as tf
import numpy as np


import matplotlib.pyplot as plt


seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

eps = 0.0001  


if not os.path.exists('./output'):
    os.mkdir('./output')

sess_path = None

data_path =None

In [ ]:
tf.disable_eager_execution()

## Initializing the Network

In [ ]:



def initialize_nn_weight(dim):
    """Initialize neural network weight or bias variable.
    Args:
        dim <list>: dimensions of weight matrix.
                    - if len(dim) == 1: initializes bias,
                    - if len(dim) == 2: initializes weight.
    Returns:
        Tensor variable of normally initialized data.
    """
    t_stnd = tf.sqrt(tf.cast(dim[0], tf.float32)) * 10
    return tf.Variable(tf.random_normal(tf.cast(dim, tf.int32)) / t_stnd, trainable=True)


def random_mini_batches(X, minibatch_size=10, seed=0):
    """Generate random minibatches from X.
    Args:
        X <array>: Input data to be mini-batched.
        minibatch_size <int>: mini-batch size.
        seed <int>: seed.
    Returns:
        List of mini-batches generated from X.
    """
    np.random.seed(seed)
    m = X.shape[0]
    mini_batches = []

    # Step 1: Shuffle X
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]

    # Step 2: Partition shuffled_X. Minus the end case.
    # number of mini batches of size mini_batch_size in your partitionning
    num_complete_minibatches = int(math.floor(m / minibatch_size))

    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[(k * minibatch_size):((k+1) * minibatch_size), :]
        mini_batch = (mini_batch_X)
        mini_batches.append(mini_batch)

    return mini_batches

In [ ]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

In [ ]:
num_episodes = 5000 
len_episodes = 10240
epochs_per_episode = 20
minibatch_size = 512
num_minibatches = int(len_episodes / minibatch_size)


# Neural network architecture parameters
num_input_nodes = 7  # Dimension of extended state space 
num_hidden_nodes = [100, 200]  # Dimension of hidden layers
num_output_nodes = (3)  # Output dimension (Investment, Mitigation and Value function)

In [ ]:
# We create a placeholder for X, the input data for the neural network, which corresponds

X = tf.placeholder(tf.float32, shape=(None, num_input_nodes))



W1 = initialize_nn_weight([num_input_nodes, num_hidden_nodes[0]])
W2 = initialize_nn_weight([num_hidden_nodes[0], num_hidden_nodes[1]])
W3 = initialize_nn_weight([num_hidden_nodes[1], num_output_nodes])

b1 = initialize_nn_weight([num_hidden_nodes[0]])
b2 = initialize_nn_weight([num_hidden_nodes[1]])
b3 = initialize_nn_weight([num_output_nodes])
# We take a softmax activiation function which is more stable for minimizing the loss
def nn_predict(X):
    hidden_layer1 = tf.nn.softmax(tf.add(tf.matmul(X, W1), b1))
    hidden_layer2 = tf.nn.softmax(tf.add(tf.matmul(hidden_layer1, W2), b2))
    output_layer = (tf.add(tf.matmul(hidden_layer2, W3), b3))
    
    return output_layer


## Model's Variable and Function

In [ ]:



# All the economics and the environmental variable and functions

t=1
psi=0.69
delta=0.0

beta=1/(1+0.15)

def utility(C):
    return ((C/L)**(1-1/psi)-1)*L/(1-1/psi)
    

l=np.float32(7403)

a=np.float32(5.115)

Ci=0.308
      
ab=0.55*1000*2.6/3.666*0.00013418
    
nE=2.6




def mitigation(m,teta=2.6):
  
    return ab*((10**(-6)+m)**teta)
    

def damage(T,p2=0.00235):
    return 1/(1+p2*(T**2))

def emissions(Y,m):
   
    return Ci*Y*(1-m)+nE
def carbon_mass(Ma,E,phi=0.9942):
    return phi*Ma+E


def carbon_mass(Ma,E,phi=0.9942):
    
    return phi*Ma+E

def temperature(T,Ma,mu0=-2.8672,mu1= 0.8954,mu2=0.4622):
    

    
    return mu0+mu1*T+mu2*tf.math.log(Ma)


def firm_net(K,m,T,A,L,alpha=0.3):
    
    env_factor=(1-mitigation(m,ab))*damage(T)
    
    r*A * K**(alpha-1)*(L/1000)**(1-alpha)

    Y = env_factor*A * K**alpha*(L/1000)**(1-alpha)
    
    return  Y,r



def firm(K,A,L,alpha=np.float32(0.3),delta=np.float32(0.)):
    
   
    r= A * K**(alpha-1)*(L/1000)**(1-alpha)+(1-delta)

    Y = A * K**alpha*(L/1000)**(1-alpha)
    
    return  Y,r
 
           

## Define Tomorrow's State Giving the Current One

In [ ]:
# Today's  state: 

K = X[:,0] # Capital
Ta = X[:,1]  # Temperature
Ma = X[:,2]  # Carbon Mass
r= X[:,3] # Cost of Capital
Yg= X[:,4] # Gross Output
L= X[:,5] # Labor Supply
A= X[:,6] # Productivity






K = tf.expand_dims(K,axis=1)
Ta =tf.expand_dims(Ta,axis=1) 
 
Ma = tf.expand_dims(Ma,axis=1)  
r= tf.expand_dims(r,axis=1) 
Yg=tf.expand_dims(Yg,axis=1)
A=tf.expand_dims(A,axis=1)
L=tf.expand_dims(L,axis=1)

#set capital greater than 100 to avoid a capital going to 0
K_modif=tf.maximum(K,(100.+eps)*tf.ones_like(K))
K_modif=tf.cast(K_modif,tf.float32)
I=nn_predict(X)[:,0]
I=tf.expand_dims(I,axis=1)
I_modif=tf.maximum(I,eps*tf.ones_like(I))
m=nn_predict(X)[:,1]
m=tf.expand_dims(m,axis=1)
V=nn_predict(X)[:,2]
V=tf.expand_dims(V,axis=1)
m_modif=tf.clip_by_value(m,eps,tf.constant(np.float32(1)))

E=emissions(Yg,m_modif)

Y_net,r_net_prime=firm_net(K_modif,m_modif,Ta,A,L)


c=Y_net-I_modif

c_modif=tf.maximum(c,tf.ones_like(c)*eps)
c_modif=tf.cast(c_modif,tf.float32)

In [ ]:

K_prime=I+(1.-delta)*K




Ma_prime=carbon_mass(Ma,E)
Ma_prime_modif=tf.maximum(Ma_prime,eps*tf.ones_like(Ma_prime))
A_prime=a*tf.ones_like(A)
L_prime=l*tf.ones_like(L)
#Same for K_prime
K_prime_modif=tf.maximum(K_prime,(100.+eps)*tf.ones_like(K_prime))
Ta_prime=temperature(Ta,Ma_prime_modif)
Ta_prime_modif=tf.maximum(Ta_prime,eps*tf.ones_like(Ta_prime))
Yg_prime,r_prime=firm(K_prime_modif,A_prime,L_prime)












#tomorrow's state
X_prime=tf.concat([K_prime_modif,Ta_prime_modif,Ma_prime_modif,r_prime,Yg_prime,L_prime,A_prime],axis=1)

out=nn_predict(X_prime)
I_prime=out[:,0]
m_prime=out[:,1]
V_prime=out[:,2]
V_prime=tf.expand_dims(V_prime,axis=1)
m_prime=tf.expand_dims(m_prime,axis=1)
m_prime_modif=tf.cast(tf.clip_by_value(m_prime,eps,np.float32(1.)),tf.float32)
I_prime=tf.expand_dims(I_prime,axis=1)
I_prime_modif=tf.maximum(I_prime,eps*tf.ones_like(I_prime))


Y_net_prime,r_net_prime=firm_net(K_prime_modif,m_prime_modif,Ta_prime,A_prime,L_prime)


c_prime=Y_net_prime-I_prime_modif
c_prime_modif=tf.maximum(c_prime,tf.ones_like(c_prime)*eps)





## Define Costs Functions

In [ ]:
# Prepare transitions to the next periods states. In this setting, there is a 25% chance
# of ending up in any of the 4 states in Z. This has been hardcoded and need to be changed
# to accomodate a different transition matrix.
beta=1/(1+0.015)


def marginal_utility(C,L,psi=0.69):
    
    return (C/L)**(-1/psi)

def utility(C,L,psi=0.69):
   
    return ((C/L)**(1-1/psi)-1)/((1-1/psi))*L


#Euler Equation

opt_euler=(beta*marginal_utility(c_modif,L)*r_net_prime/marginal_utility(c_prime_modif,L) -1)

#punish for negative capital
opt_punish_capital =tf.maximum(100.-K_prime, tf.zeros_like(K_prime))

opt_punish_capital = tf.cast(opt_punish_capital,tf.float32)

#punish for negative consumption
opt_punish_conso =  tf.maximum(- c, tf.zeros_like(c))
opt_punish_conso_prime = tf.maximum(-c_prime, tf.zeros_like(c_prime))

#punish for negative investment
opt_punish_I =  tf.maximum(- I, tf.zeros_like(I))
opt_punish_I_prime = tf.maximum(-I_prime, tf.zeros_like(I_prime))
  
#punish for decreasing capital 

opt_punish_incr =  tf.maximum(K_modif-K_prime_modif, tf.zeros_like(I))


#punish if the mitigation is not below 0 and 1
opt_punish_mu0 = tf.maximum(- m_prime, tf.zeros_like(m_prime))
opt_punish_mu0 = tf.cast(opt_punish_mu0,tf.float32)



    
opt_punish_mu1 =  tf.maximum( m_prime-1, tf.zeros_like(m_prime))
opt_punish_mu1 = tf.cast(opt_punish_mu1,tf.float32)


opt_punish_mu0_or =tf.maximum(- m, tf.zeros_like(m))
opt_punish_mu0_or = tf.cast(opt_punish_mu0_or,tf.float32)

    
opt_punish_mu1_or =  tf.maximum( m-1, tf.zeros_like(m))
opt_punish_mu1_or = tf.cast(opt_punish_mu1_or,tf.float32)

#punish for negative temperature    
opt_punish_T =  tf.maximum( -Ta_prime, tf.zeros_like(Ta_prime))

#Bellman Equation for the optimal path    
opt_bell=((utility(c_modif,L)+beta*V_prime)/V)-1


# We combine them and use then MSE for optimizer
combined_opt = [ opt_punish_incr,opt_bell,opt_punish_capital,
                opt_punish_I,opt_punish_I_prime ,opt_punish_conso, 
                opt_punish_conso_prime,opt_punish_mu0,opt_punish_mu1,opt_punish_mu0_or,opt_punish_mu1_or
                           ]
               
               
opt_predict = tf.concat(combined_opt, axis=1)



opt_correct = tf.zeros_like(opt_predict)


## Optimizer & Training Step


In [ ]:
# issue for clipping gradient with some None value
def ClipIfNotNone(grad):
    if grad is None:
        return grad
    return tf.clip_by_value(grad, -1, 1)

  

In [ ]:
# Define the cost function
cost = tf.losses.mean_squared_error(opt_correct, opt_predict)

# Adam optimizer with a expenotential decaying learning rate because it is more stable
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 1.
decayed_lr = tf.train.exponential_decay(starter_learning_rate,
                                        global_step, 10000,
                                        0.2, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate= decayed_lr)


gvs = optimizer.compute_gradients(cost)
#capped_gvs = [(ClipIfNotNone(grad), var) for grad, var in gvs]
  

# Define a training step
train_step = optimizer.apply_gradients(gvs)


In [ ]:
#we simulate the future states given the neural network and the first state


def simulate_episodes(sess, x_start, episode_length, print_flag=True):

    time_start = datetime.now()
    if print_flag:
        print('Start simulating {} periods.'.format(episode_length))
    dim_state = np.shape(x_start)[1]

    X_episodes = np.zeros([episode_length, dim_state])
    X_episodes[0, :] = x_start
    X_old = x_start
    

   
    

    for t in range(1,episode_length):
        
        X_new = sess.run(X_prime, feed_dict={X: X_old})
        
        
        # Append it to the dataset
        X_episodes[t, :] = X_new
    
        X_old = (X_new)
       

    time_end = datetime.now()
    time_diff = time_end - time_start
    if print_flag:
        print('Finished simulation. Time for simulation: {}.'.format(time_diff))

    return X_episodes

## Training the model



In [ ]:


if data_path:
    X_data_train = np.load(data_path)
    print('Loaded initial data from ' + data_path)
    start_episode = int(re.search('_(.*).npy', data_path).group(1))
else:
    # generate a random starting point with only positive values
    X_data_train = np.random.uniform(1,2,(1,7))
 
    X_data_train=np.float32(X_data_train)
    
  
    start_episode = 0
    
if sess_path is not None:
    saver.restore(sess, sess_path)
   

    train_seed = 0

init = tf.global_variables_initializer()

saver = tf.train.Saver()

sess.run(init)


            
for episode in range(start_episode, 5000):
    
    

    X_episodes= simulate_episodes(sess, np.float32(X_data_train), 1000, print_flag=(episode==0))

    X_data_train=X_episodes[-1:,]
    
    for epoch in range(epochs_per_episode):

        train_seed += 1
        minibatch_cost = 0

       
        minibatches = random_mini_batches(X_episodes, 30, train_seed)

        for minibatch_X in minibatches:
         
            minibatch_cost += sess.run(cost, feed_dict={X: minibatch_X}) / num_minibatches
           
              
      

        for minibatch_X in minibatches:
           
            sess.run(train_step, feed_dict={X: minibatch_X})
            cost_store.append(minibatch_cost)


  
   
    print('Episode {}: Cost: {:.4f}'.format(episode,minibatch_cost ))
                                                                                   
                                                                                  

    if episode%100==0:    

        # Save the tensorflow session
        saver.save(sess, './output/sess_{}.ckpt'.format(episode))
        # Save the starting point
        np.save ('./output/data_{}.npy'.format(episode), X_data_train)                                                                          
                                                                                 